In [10]:
write_html = False 

In [1]:
#%%capture
%run ../m_ee_fit/input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 10

In [3]:
centralities = ["0-93%","0-20%","20-40%","40-60%","60-93%","40-93%"]
Opt_names = ["loose", "basic","tight"]
Hists_names = ["m_{ee}_stat","m_{ee}_syst","dca_stat","dca_syst","dca_{J/#Psi}_stat","dca_{J/#Psi}_syst"]
part_names = ["#pi^{0} #rightarrow ee#gamma","#phi #rightarrow ee","J/#Psi","c#bar{c}","b#bar{b}","#omega #rightarrow ee","QGP","#pi^{-}","#gamma #rightarrow ee"]

In [4]:
N_centr = 6
N_options = 6
N_pt = 1
colors=[2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kBlack,root.kCyan,root.kYellow,root.kGreen,root.kOrange]
central_bins = [0,20,60,80,20,40,40,60,60,80,80,100]
pt_boarders = [0.4,0.7,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
sim_file_path="../sim/output/Newembed/invmass/decomposition/"
sim_file_names=['dalitz_50M_v2.root', 'phi_50M_v1.root', 'jpsi_50M_v1.root', 'ccbar_50M_v2.root', 'bbbar_10M_v1.root', 'omega_10M_v0.root', 'thermal_25M_200_v0.root', 'bbbar_10M_v0.root']
hist_select_3D_names = ["inv_mass_dca_fg1_6","inv_mass_dca_fg1_5","inv_mass_dca_fg1_6","inv_mass_dca_fg1_7","inv_mass_dca_fg1_8","inv_mass_dca_fg1_7",\
                        "inv_mass_dca_fg2_6","inv_mass_dca_fg2_5","inv_mass_dca_fg2_6","inv_mass_dca_fg2_7","inv_mass_dca_fg2_8","inv_mass_dca_fg2_7",\
                        "inv_mass_dca_fg4_6","inv_mass_dca_fg4_5","inv_mass_dca_fg4_6","inv_mass_dca_fg3_7","inv_mass_dca_fg3_8","inv_mass_dca_fg3_7",\
                        "inv_mass_dca_gen_6","inv_mass_dca_gen_5","inv_mass_dca_gen_6","inv_mass_dca_gen_7","inv_mass_dca_gen_8","inv_mass_dca_gen_7"]
N_hists = len(hist_select_3D_names)

In [5]:
dNdy = [41.4,0.421,0.759/1000.,0.12,0.01,3.65,10,0.133/1000]
Ncolls = [251,771,282.4,82.6,12.1,40]
Br_to_ee = [1.174e-2, 2.97e-4, 5.94e-2, 0.01, 0.01, 7.28e-5, 1e-2, 7.9e-3]
RAA = [[0.4,0.8,0.4,0.4,0.4,0.4,0.4,0.4],[0.3,0.6,0.3,0.3,0.3,0.3,0.3,0.3],[0.4,0.8,0.4,0.4,0.4,0.4,0.4,0.4],[0.8,1.2,0.8,0.8,0.8,0.8,0.8,0.8],[0.9,1.0,0.9,0.9,0.9,0.9,0.9,0.9],[0.9,1.1,0.8,0.9,0.9,0.9,0.9,0.9]]
dNdy_centr = [[dNdy[i]*Ncolls[j]/42.2*RAA[j][i] for i in range(8)] for j in range(6)]

In [6]:
hists_sim = []

for iFile in range(len(sim_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
        if ihist%6==0:hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}_{1}")
        if ihist%6==5:hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}_{0}")
    hists_sim.append(hists_sim_file)
    infile.Close()

In [11]:
func_array = []
for icentr in range(N_centr):
    func_array.append([])
    for isim in range(len(sim_file_names)-1):
        func_array[-1].append(root.TF1(f"func_{icentr}_{isim}", "[3]*x * (1 + ([1]-1) * (sqrt(x*x +[0]*[0]) - [0]) / ([1] * [2]))**(-[1])", 0.2, 10.0))
        func_array[-1][-1].SetParameters(0.13957, 5.0, 0.1,2),  # Example parameters: mass, n, T
        func_array[-1][-1].SetParLimits(0, 0.1, 4.2)
        func_array[-1][-1].SetParLimits(1, 4, 12)
        func_array[-1][-1].SetParLimits(2, 0.1, 0.9)

In [13]:
func_array[0][0].SetParameter(0,3.2)
print(func_array[0][0].Integral(0.2,10.0))

0.16714198508200387


In [8]:
c0 = root.TCanvas(f"c0",f"c0",1500,1400)
c0.Divide(3,2)
iOpt = 0
legends =[]
for icentr in range(0,N_centr):
    legends.append(root.TLegend(0.6,0.6,0.99,0.99,centralities[icentr]))
    c0.cd(icentr+1)
    root.gPad.SetLogy()

    for isim in range(0,len(sim_file_names)-1):
        sim_proj = hists_sim[isim][icentr+18].ProjectionZ(hists_sim[isim][icentr+18].GetName()+"_proj",1,-1,1,-1) 
        if isim == 0:
            Format_Hist_total(sim_proj,"p_{T}, GeV","dN/dp_{T}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,1,3,1,0.85,1.24)
            sim_proj.SetMaximum(1e9)
            sim_proj.SetMinimum(1e-0)
        Format_Graph(sim_proj,21+isim,0,colors[isim],3,colors[isim])
        sim_proj.Draw("same H")
        legends[-1].AddEntry(sim_proj,part_names[isim],"l")
        sim_proj.Fit(func_array[icentr][isim],"QR")

    #legends[-1].AddEntry(0,"","a")
    Format_Legend(legends[-1],0.08,12)
    legends[-1].Draw()

c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr.png")

In [9]:
print(func_array[2][2].GetParameter(3),func_array[2][2].Integral(0.2,10.0))

924765.338929664 1737655.7362493807
